In [9]:
import ee
import geemap
import math
from datetime import datetime

In [10]:
ee.Authenticate()
ee.Initialize()
Map = geemap.Map()

In [13]:
# Constants
gsc = ee.Number(0.0820) # solar constant MJ/m2
# Regression constant, intercept, expressing the fraction of extra-terrestrial radiation reaching the earth on overcast days
As = ee.Number(0.23)
Bs = ee.Number(0.5)


# Define the year and month
year = 2010
month = 2 
day = 1

# Define start and end dates
start_date = ee.Date.fromYMD(year, month, 1)  # First day of the month
end_date = start_date.advance(1, 'month')    # Advance by one month

# Date of Year (DOY)
j = datetime(year, month, day).timetuple().tm_yday 


# Define region of interest (roi)
roi = ee.FeatureCollection('projects/ee-athiraganesan314/assets/Shivalik_range')

# Define roi for sampling 
aoi_minmax = ee.FeatureCollection('users/athiraganesan314/aoi_minmax')

# Tower location for Sampling
pts = ee.FeatureCollection('projects/ee-athiraganesan314/assets/tower_locations')

Map.centerObject(roi,12)
# Map.addLayer(roi,{},"roi")

# radians
lonlat = ee.Image.pixelLonLat().clip(roi);
radians = lonlat.expression('(b(1) * 3.141592653589793) / 180', 
{'b': lonlat.select(['latitude'])});

# Load and process MODIS LST data
lst = (ee.ImageCollection('MODIS/061/MOD11A1')
       .filterDate(start_date, end_date)
       .filterBounds(roi)
       .select('LST_Day_1km')
       .mean()
       .clip(roi)
       .multiply(0.02))  # Convert to Celsius

# Load and process NDVI data
ndvi = (ee.ImageCollection('MODIS/061/MOD13A1')
        .filterDate(start_date, end_date)
        .filterBounds(roi)
        .select('NDVI')
        .median()
        .clip(roi)
        .multiply(0.0001))  # Scale factor

# Load and process ERA5 wind data
wind = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
        .filterDate(start_date, end_date)
        .filterBounds(roi)
        .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])
        .first()
        .clip(roi))

# Surface reflectance dataset and albedo calculation
dataset = (ee.ImageCollection('MODIS/061/MOD09GA')
           .filterDate(start_date, end_date)
           .filterBounds(roi))
bands_alb = (dataset.select(['sur_refl_b01', 'sur_refl_b02'])
             .mean()
             .multiply(0.0001)
             .clip(roi))
alb = bands_alb.expression(
    '0.08 + (0.41 * RED) + (0.14 * NIR)', {
        'NIR': bands_alb.select('sur_refl_b02'),
        'RED': bands_alb.select('sur_refl_b01')
    })

# Load and process Temp data
temp = (ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
        .filterDate(start_date, end_date)
        .filterBounds(roi)
        .select('temperature_2m')
        .first()
        .clip(roi))

# Load and process elevation data
z = (ee.Image('USGS/SRTMGL1_003')
        .select('elevation')
        .clip(roi));

# Sample Min and Max Temperature 
temp_minmax = temp.reduceRegions(
collection = aoi_minmax,
reducer = ee.Reducer.minMax()
)
stack=wind.addBands(alb) 
sampled= stack.sampleRegions(
  collection = pts,
  properties = ['point'],
  scale = 9000,
  geometries = True
  
)

# Extract the values of some environmental variable(wind,albedo,temperature)
mete_values = ee.FeatureCollection([sampled,temp_minmax]).flatten()

uwind= mete_values.aggregate_mean('u_component_of_wind_10m') # U component of wind
vwind= mete_values.aggregate_mean('v_component_of_wind_10m') # V component of wind
dew= mete_values.aggregate_mean('dewpoint_temperature_2m')   # Dewpoint temperature
alb= mete_values.aggregate_mean('constant') # Albedo
tmax= mete_values.aggregate_mean('max') # Max Temperature
tmin= mete_values.aggregate_mean('min') # Min Temperature

# Functions for calculating atmospheric and environmental variable for the calculation of ETo and ETa

def calculate_kc(lst, ndvi, alb):
    return lst.subtract(273.15).divide(ndvi.multiply(alb)).multiply(-0.008).add(1.8).exp()

def calculate_tmean(tmin, tmax):
    return (tmin.add(tmax)).divide(2)

def calculate_uh(uwind, vwind):
    return ((uwind.pow(2)).add(vwind.pow(2))).sqrt()

def calculate_u2(uh):
    return uh.multiply(0.7479511)

def calculate_ssvp(tmean):
    return ((((((tmean.subtract(273.15)).multiply(17.27)).divide(((tmean.subtract(273.15)).add(237.3)))).exp())
             .multiply(0.6108)).multiply(4098)).divide((((tmean.subtract(273.15)).add(237.3)).pow(2)))

def calculate_p(z):
    return ((((ee.Number(293)).subtract(z.multiply(0.0065))).divide(ee.Number(293))).pow(ee.Number(5.26))).multiply(101.3)

def calculate_r(p):
    return p.multiply(0.000665)

def calculate_etmax(tmax):
    return tmax.subtract(273.15).multiply(17.27).divide(
        tmax.subtract(273.15).add(237.3)
    ).exp().multiply(0.6108)

def calculate_etmin(tmin):
    return tmin.subtract(273.15).multiply(17.27).divide(
        tmin.subtract(273.15).add(237.3)
    ).exp().multiply(0.6108)  

def calculate_ea(tmin):
    return tmin.subtract(273.15).multiply(17.27).divide(
        tmin.subtract(273.15).add(237.3)
    ).exp().multiply(0.6108)


def calculate_es(etmax, etmin):
    return (etmax.add(etmin)).divide(2)

def calculate_dr(j):
    return (((ee.Number(((math.pi*2)/365)*j)).cos()).multiply(0.033)).add(1)

def calculate_sd(j):
    return (ee.Number((((math.pi*2)/365)*j)-(1.39)).sin()).multiply(0.409)

def calculate_ws(radians):
    return ((((radians).multiply(-1)).tan()).multiply(sd.tan())).acos()

def calculate_Ra(ws, radians, sd, gsc, dr):
    return ((ws.multiply(ee.Number(radians).sin()).multiply(sd.sin())).add((ee.Number(radians).cos()).multiply(sd.cos()).multiply(ws.sin()))).multiply(dr).multiply(gsc).multiply(1440/(math.pi))

def calculate_Rso(Ra):
    return Ra.multiply(0.759)

def calculate_N(ws):
    return (ee.Number(24).divide(math.pi)).multiply(ws)

def calculate_n(tmean):
    return tmean.subtract(273.15).multiply(0.232).add(4.352)

def calculate_Rs(n, N, As, Bs, Ra):
    return (ee.Number(n.divide(N))).multiply(Bs).add(As.multiply(Ra))

def calculate_Rns(alb, Rs):
    return ((ee.Number(1)).subtract(alb)).multiply(Rs)

def calculate_Rnl(tmax, tmin, ea, Rs, Rso):
    return ((((((tmax.subtract(273.15)).add(273.16)).pow(4)).add(((tmin.subtract(273.15)).add(273.16)).pow(4))).divide(2)).multiply(0.000000004903)).multiply(ee.Number(0.34).subtract((ea.sqrt().multiply(0.14)))).multiply(ee.Number(1.35).multiply(Rs.divide(Rso)).subtract(0.35))

def calculate_Rn(Rns, Rnl):
    return Rns.subtract(Rnl)

def calculate_ETo(Rn, ssvp, tmean, u2, r, ea, es):
        # Convert mean temperature to Celsius
    tmean_celsius = tmean.subtract(273.15)

    # First term
    term1 = ee.Number(0.408).multiply(Rn).multiply(ssvp).divide(
        u2.multiply(0.34).add(1).multiply(r).add(ssvp)
    )

    # Second term
    term2 = ee.Number(900).divide(tmean_celsius.add(273)).multiply(u2).multiply(
        es.subtract(ea)
    ).multiply(r).divide(
        u2.multiply(0.34).add(1).multiply(r).add(ssvp)
    )

    # Sum the terms
    return term1.add(term2)

   
def calculate_ETa(kc, ETo):
    return ETo.multiply(kc)


# Calculate crop coefficient (kc)
kc = calculate_kc(lst, ndvi, alb)

# Calculate other atmospheric and environmental variable for the calculation of ETo and ETa
tmean = calculate_tmean(tmin,tmax) # Mean Temperature
uh = calculate_uh(uwind, vwind) # wind speed
u2 = calculate_u2(uh) # wind speed at 2 m 
ssvp = calculate_ssvp(tmean) # Slope of Saturation Vapor Pressure Curve
p = calculate_p(z) # atmospheric pressure 
r = calculate_r(p) # Psychrometric constant
etmax = calculate_etmax(tmax) # saturation vapor pressure at the max temperature
etmin = calculate_etmin(tmin) # saturation vapor pressure at the min temperature 
ea = calculate_ea(tmin) # actual vapor pressure
es = calculate_es(etmax, etmin) # saturation vapor pressure
dr = calculate_dr(j) # inverse relative distance Earth-Sun
sd = calculate_sd(j) # solar declination 
ws = calculate_ws(radians) #  solar hour angle
Ra = calculate_Ra(ws, radians, sd, dr, gsc) # extraterrestrial radiation 
Rso = calculate_Rso(Ra) # Clear sky solar radiation
N = calculate_N(ws) # maximum possible duration of sunshine
n = calculate_n(tmean) # ctual duration of sunshine
Rs = calculate_Rs(n, N, Bs, As, Ra) # Mean daily solar radiation 
Rns = calculate_Rns(alb, Rs) # Net solar or net shortwave radiation
Rnl = calculate_Rnl(tmax, tmin, ea, Rs, Rso) # extraterrestrial radiation
Rn = calculate_Rn(Rns, Rnl) # Net Radiation

# Reference Evapotranspiration (FAO 56)
ETo = calculate_ETo(Rn, ssvp, tmean, u2, r, ea, es)


# Actual Evapotranspiration
ETa = (kc, ETo)


Map.addLayer(ETo,{},"ETo")
display(Map)



Map(bottom=430327.0, center=[30.84969750340992, 77.24333304891434], controls=(WidgetControl(options=['position…